<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</td>
</table>

# Using the Neo4j Aura API

Let's begin by installing all the needed libraries

In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      /private/var/folders/9c/x0wy8bk13c18rzl1r14bg0vw0000gn/T/pip-install-8rrx3zpr/pandas_c32ecccf7b9c4a4c9c0d4bf734ccf667/setup.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        import pkg_resources
      /private/var/folders/9c/x0wy8bk13c18

In [3]:
from dotenv import dotenv_values
from urllib.parse import urljoin
import subprocess
import datetime
import json

## Step 1: Authenticate API and get a bearer token for access

Upload the API credential file you created earlier. If you are using Colab you can click the button below to locate and upload the file. If you are using another platform you can do this manually. 

In [1]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

Now we'll extract the credentials from the file into a dictionary for further use. 

In [ ]:
filename = list(uploaded.keys())[0]
print(filename)

aura_creds = dotenv_values(filename)
for item in aura_creds:
  print(item, '=', aura_creds[item])

Let's define a helper function which uses these credentials to generate a bearer token for access.

In [ ]:
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = 'api_bearer_token'
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    return access_token 

Now let's try it.

In [ ]:
api_base = 'https://api.neo4j.io/'

access_token = refresh_token(aura_creds, api_base)
print('access_token:', access_token)

## Step 2: Explore the Aura tenants

Now that we've authenticated, let's take a look around and see what we have access to.

Let's define a few helper functions.

In [ ]:
def list_tenants(access_token, api_base, tenant_id=None):
    #api_endpoint = 'https://api.neo4j.io/v1/tenants'

    api_endpoint = urljoin(api_base, '/v1/tenants')

    aura_tenants = {}

    if tenant_id:
        api_endpoint += '/' + tenant_id

    print()
    print('api_endpoint:', api_endpoint)
    print()

    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    
    list_tenants = dict(json.loads(subprocess.check_output(list_cmd, shell=True)))['data']
    for item in list_tenants:
        aura_tenants[item['id']] = {}
        aura_tenants[item['id']]['id'] = item['id'] 
        aura_tenants[item['id']]['name'] = item['name'] 

    return aura_tenants


def tenant_info(access_token, api_base, tenant_id):
    api_endpoint = urljoin(api_base, '/v1/tenants/' + tenant_id)
    tenant_data = {}
    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    tenant_info = json.loads(subprocess.check_output(info_cmd, shell=True))['data']
    
    for item in tenant_info:
        #print(item, '=', tenant_info[item])
        tenant_data[tenant_info['id']] = {}
        tenant_data[tenant_info['id']]['tenant_id'] = tenant_info['id']
        tenant_data[tenant_info['id']]['tenant_name'] = tenant_info['name'] 
        tenant_data[tenant_info['id']]['instance_configurations'] = list(tenant_info['instance_configurations'])

    return tenant_data


Now let's take a look at the environment, beginning with a list of tenants we have access to.

In [ ]:
aura_tenants = list_tenants(access_token, api_base)
print()
print('Aura Tenants:')
print('-------------')
for tenant in aura_tenants:
    print(aura_tenants[tenant])
print()

We can also get a take a more detailed look into one of the tenants.

In [ ]:
sample_tenant_id = next(iter(aura_tenants))
print('sample_tenant_id:', sample_tenant_id)

tenant_details = tenant_info(access_token, api_base, sample_tenant_id)

print('Tenant Details:')
print('---------------')
for item in tenant_details[sample_tenant_id]:
    print(item, '=', tenant_details[sample_tenant_id][item])